In [1]:
import pandas as pd
%cd ..

C:\Users\georg\Documents\msc-projcet


In [95]:
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate,GridSearchCV
from sklearn import svm,tree,naive_bayes,linear_model,dummy,neural_network,ensemble
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [2]:
df = pd.read_csv("data/interim/tracks.csv")

In [3]:
df.head()

,id,title,album,views,videos,tt_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,1sx0XsWUusUoPstbvokZjP,A-O-K,A-O-K,24500000,21,6953851540646709249,0.731,0.626,11,-6.919,...,0.000000,0.1830,0.862,82.488,audio_features,spotify:track:1sx0XsWUusUoPstbvokZjP,https://api.spotify.com/v1/tracks/1sx0XsWUusUo...,https://api.spotify.com/v1/audio-analysis/1sx0...,173640,4
1,6n4U3TlzUGhdSFbUUhTvLP,Walk It Talk It,Culture II,15000,2,6790838366146398209,0.907,0.633,2,-5.145,...,0.000003,0.1060,0.395,145.914,audio_features,spotify:track:6n4U3TlzUGhdSFbUUhTvLP,https://api.spotify.com/v1/tracks/6n4U3TlzUGhd...,https://api.spotify.com/v1/audio-analysis/6n4U...,276147,4
2,4jDmJ51x1o9NZB5Nxxc7gY,Careless Whisper,Heart & Soul,12000000,4,6712428451149645825,0.574,0.629,2,-8.815,...,0.000000,0.2710,0.786,153.119,audio_features,spotify:track:4jDmJ51x1o9NZB5Nxxc7gY,https://api.spotify.com/v1/tracks/4jDmJ51x1o9N...,https://api.spotify.com/v1/audio-analysis/4jDm...,300107,4
3,2P5yIMu2DNeMXTyOANKS6k,Nobody,Be the Cowboy,245100,1,6754663153281321734,0.398,0.717,0,-5.253,...,0.000000,0.0850,0.487,117.296,audio_features,spotify:track:2P5yIMu2DNeMXTyOANKS6k,https://api.spotify.com/v1/tracks/2P5yIMu2DNeM...,https://api.spotify.com/v1/audio-analysis/2P5y...,193429,4
4,1Wdj4wRDYS7aT4CoPS0mAH,You Are Enough,Atlas: I,7900000,15,6779450671369291777,0.593,0.500,1,-8.532,...,0.000012,0.0837,0.465,117.996,audio_features,spotify:track:1Wdj4wRDYS7aT4CoPS0mAH,https://api.spotify.com/v1/tracks/1Wdj4wRDYS7a...,https://api.spotify.com/v1/audio-analysis/1Wdj...,180087,4


In [4]:
df.describe()

,views,videos,tt_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,2.187000e+03,2187.000000,2.187000e+03,2187.000000,2187.000000,2187.000000,2187.000000,2187.000000,2187.000000,2187.000000,2187.000000,2187.000000,2187.000000,2187.000000,2187.000000,2187.000000
mean,6.696997e+06,5.841792,6.381853e+18,0.643339,0.615903,5.303612,-8.172599,0.556927,0.111113,0.299595,0.144327,0.179097,0.513346,121.438764,199452.427984,3.933699
std,1.785776e+07,14.513099,1.585278e+18,0.180653,0.227732,3.608240,4.881110,0.496862,0.117416,0.308458,0.305159,0.147594,0.257360,29.390177,73237.552465,0.390525
min,0.000000e+00,1.000000,6.557410e+05,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8500.000000,0.000000
25%,5.785000e+04,1.000000,6.726731e+18,0.542000,0.465000,2.000000,-9.633500,0.000000,0.039200,0.039250,0.000000,0.092400,0.308500,98.968000,158169.500000,4.000000
50%,1.597000e+05,3.000000,6.808590e+18,0.674000,0.653000,5.000000,-6.948000,1.000000,0.059300,0.181000,0.000013,0.121000,0.517000,121.876000,196364.000000,4.000000
75%,2.700000e+06,5.000000,6.929740e+18,0.774000,0.794000,8.000000,-5.188000,1.000000,0.132500,0.501500,0.015400,0.215000,0.722500,139.933000,234373.000000,4.000000
max,2.057000e+08,289.000000,7.013915e+18,0.979000,1.000000,11.000000,-0.949000,1.000000,0.935000,0.996000,1.000000,0.979000,1.000000,220.099000,998000.000000,5.000000


In [14]:
metric = r2_score
std = StandardScaler() 
nrm = MinMaxScaler()

,danceability,energy,key,loudness,mode,instrumentalness,liveness,valence,tempo,views
0,0.731,0.626,11,-6.919,1,0.000000,0.1830,0.862,82.488,24500000
1,0.907,0.633,2,-5.145,1,0.000003,0.1060,0.395,145.914,15000
2,0.574,0.629,2,-8.815,0,0.000000,0.2710,0.786,153.119,12000000
3,0.398,0.717,0,-5.253,1,0.000000,0.0850,0.487,117.296,245100
4,0.593,0.500,1,-8.532,1,0.000012,0.0837,0.465,117.996,7900000


In [32]:
data = df.loc[:,['danceability','energy','key','loudness','mode','instrumentalness','liveness','valence','tempo','views']]
targets = data['views'].apply(np.digitize,bins=[0,1e+5,1e+6])
data = data.loc[:,['danceability','energy','key','loudness','mode','instrumentalness','liveness','valence','tempo']]

In [74]:
regularized = nrm.fit_transform(data)
X, y = regularized, targets

In [67]:
def evaluate_model(model,X,y):
    scoring = ['precision_macro','recall_macro','accuracy']
    scores = cross_validate(model,X,y,scoring=scoring)
    mean_precision = scores['test_precision_macro'].mean()
    mean_recall = scores['test_recall_macro'].mean()
    mean_accuracy = scores['test_accuracy'].mean()
    return mean_precision,mean_recall,mean_accuracy

In [94]:
models = [
    ("Baseline Classifier",dummy.DummyClassifier(strategy='stratified')),
    ("Decision Tree",tree.DecisionTreeClassifier()),
    ("Random Forest",ensemble.RandomForestClassifier()),
    ("SVC",svm.SVC()),
    ("Logistic Regression",linear_model.LogisticRegression()),
    ("Complement Naive Bayes",naive_bayes.ComplementNB()),
    ("Multi-Layer Perceptron",neural_network.MLPClassifier(random_state=1,max_iter=400,hidden_layer_sizes=(256,128)))
]
template = "{0:<25} {1:<4.2} {2:<4.2} {3:<4.2}"
print(str.format(template,"Classifier","Precision","Recall","Accuracy"))
for name, model in models:
    scores = evaluate_model(model,X,y)
    print(str.format(template,name,*scores))

Classifier                Pr   Re   Ac  
Baseline Classifier       0.33 0.33 0.33
Decision Tree             0.32 0.32 0.32
Random Forest             0.33 0.33 0.35
SVC                       0.38 0.34 0.4 
Logistic Regression       0.32 0.33 0.39
Complement Naive Bayes    0.35 0.35 0.36


c:\users\georg\documents\msc-projcet\venv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\georg\documents\msc-projcet\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\users\georg\documents\msc-projcet\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


Multi-Layer Perceptron    0.36 0.36 0.37


In [157]:
params = [
    {'C':[0.1,1,10],'kernel':['linear']},
    {'C':[0.1,1,10],'gamma':['scale',0.1,0.5],'kernel':['rbf','sigmoid']},
    {'C':[0.1,1,10],'gamma':['scale',0.1,0.5,0.9],'kernel':['poly'],'degree':[3,4]}
]
optimizer = GridSearchCV(
    svm.SVC(),
    param_grid=params,
    scoring='precision_macro',
    n_jobs=3,
    refit=True,
    verbose=4
)
optimizer.fit(X,y)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


GridSearchCV(estimator=SVC(), n_jobs=3,
             param_grid=[{'C': [0.1, 1, 10], 'kernel': ['linear']},
                         {'C': [0.1, 1, 10], 'gamma': ['scale', 0.1, 0.5],
                          'kernel': ['rbf', 'sigmoid']},
                         {'C': [0.1, 1, 10], 'degree': [3, 4],
                          'gamma': ['scale', 0.1, 0.5, 0.9],
                          'kernel': ['poly']}],
             scoring='precision_macro', verbose=4)

In [151]:
pd.DataFrame(optimizer.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,param_gamma,param_degree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.082533,0.003875,0.015079,0.000506,0.1,linear,NaN,NaN,"{'C': 0.1, 'kernel': 'linear'}",0.189542,0.190321,0.190632,0.190632,0.189853,0.190196,0.000434,33
1,0.082137,0.001239,0.014880,0.000543,1,linear,NaN,NaN,"{'C': 1, 'kernel': 'linear'}",0.189542,0.190321,0.190632,0.190632,0.189853,0.190196,0.000434,33
2,0.091561,0.001949,0.014583,0.000243,10,linear,NaN,NaN,"{'C': 10, 'kernel': 'linear'}",0.189542,0.190321,0.190632,0.190632,0.189853,0.190196,0.000434,33
3,0.138681,0.001818,0.067158,0.000506,0.1,rbf,scale,NaN,"{'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}",0.189542,0.190321,0.190632,0.190632,0.189853,0.190196,0.000434,33
4,0.206236,0.012717,0.036704,0.001775,0.1,sigmoid,scale,NaN,"{'C': 0.1, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.188287,0.190164,0.195414,0.190164,0.189853,0.190776,0.002421,32
5,0.152272,0.002717,0.073904,0.004403,0.1,rbf,0.1,NaN,"{'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}",0.189542,0.190321,0.190632,0.190632,0.189853,0.190196,0.000434,33
6,0.140268,0.007485,0.021725,0.000578,0.1,sigmoid,0.1,NaN,"{'C': 0.1, 'gamma': 0.1, 'kernel': 'sigmoid'}",0.189542,0.190321,0.190632,0.190632,0.189853,0.190196,0.000434,33
7,0.139673,0.002781,0.067952,0.001537,0.1,rbf,0.5,NaN,"{'C': 0.1, 'gamma': 0.5, 'kernel': 'rbf'}",0.189542,0.190321,0.190632,0.190632,0.189853,0.190196,0.000434,33
8,0.190464,0.004150,0.033529,0.000397,0.1,sigmoid,0.5,NaN,"{'C': 0.1, 'gamma': 0.5, 'kernel': 'sigmoid'}",0.199425,0.190632,0.195414,0.190944,0.189853,0.193254,0.003648,31
9,0.143840,0.002008,0.066662,0.000920,1,rbf,scale,NaN,"{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}",0.237505,0.272445,0.264790,0.257993,0.237978,0.254142,0.014151,20


In [158]:
optimizer.best_params_

{'C': 1, 'gamma': 0.5, 'kernel': 'rbf'}

In [159]:
est = optimizer.best_estimator_

In [160]:
evaluate_model(est,X,y)

c:\users\georg\documents\msc-projcet\venv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\georg\documents\msc-projcet\venv\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4083569438961784, 0.340217706385673, 0.40146599375150205)